#### Libraries Used

In [1]:
import fingerprint_feature_extractor
import fingerprint_enhancer
import cv2
import os
import csv

"""
https://github.com/Utkarsh-Deshmukh/Fingerprint-Feature-Extraction
install with 'pip install fingerprint-feature-extractor'

https://github.com/Utkarsh-Deshmukh/Fingerprint-Enhancement-Python
install with 'pip install fingerprint_enhancer'
"""

"\nhttps://github.com/Utkarsh-Deshmukh/Fingerprint-Feature-Extraction\ninstall with 'pip install fingerprint-feature-extractor'\n\nhttps://github.com/Utkarsh-Deshmukh/Fingerprint-Enhancement-Python\ninstall with 'pip install fingerprint_enhancer'\n"

#### Function Definitions

In [2]:
def pad_or_truncate_minutiae_list(minutiaeList, targetLength):
    # Define dummy minutiae with impossible coordinates and orientation as placeholder
    dummy_minutiae = fingerprint_feature_extractor.MinutiaeFeature(-1, -1, float('nan'), 'Dummy')
    currentLength = len(minutiaeList)
    
    if currentLength > targetLength:
        return minutiaeList[:targetLength]
    else:
        return minutiaeList + [dummy_minutiae] * (targetLength - currentLength)

def process_images_from_folder(folder_path, csv_filepath, target_minutiae_count, enrollment = True):
    headers = ['ID', 'Type', 'LocationX', 'LocationY', 'Orientation']
    
    with open(csv_filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        ID = 0
        for filename in os.listdir(folder_path):
            if filename.endswith(".tif"):  # Check for TIFF images, adjust if using other formats
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path, 0)
                out = fingerprint_enhancer.enhance_Fingerprint(img)
                
                # Process image to enhance and extract features, replace with your actual method
                # out = fingerprint_enhancer.enhance_Fingerprint(img)
                if(enrollment==False):
                    cv2.imshow('enhanced_image', out);
                    cv2.waitKey(0)
                    FeaturesTerminations, FeaturesBifurcations = fingerprint_feature_extractor.extract_minutiae_features(
                                    out, spuriousMinutiaeThresh=10, invertImage=False, showResult=True, saveResult=True)
                else:
                    FeaturesTerminations, FeaturesBifurcations = fingerprint_feature_extractor.extract_minutiae_features(
                                    out, spuriousMinutiaeThresh=10, invertImage=False, showResult=False, saveResult=True)
                
                # Ensure a fixed number of minutiae
                FeaturesTerminations = pad_or_truncate_minutiae_list(FeaturesTerminations, target_minutiae_count//2)
                FeaturesBifurcations = pad_or_truncate_minutiae_list(FeaturesBifurcations, target_minutiae_count//2)

                # Write termination features
                for feature in FeaturesTerminations:
                    writer.writerow([ID, 0, feature.locX, feature.locY, feature.Orientation])
                
                # Write bifurcation features
                for feature in FeaturesBifurcations:
                    writer.writerow([ID, 1, feature.locX, feature.locY, feature.Orientation])
            ID+=1

#### Write .csv from fingerprint data

In [ ]:

folder_path = r"C:\Users\quinn\Desktop\CryptoProj\SampleSet"
csv_filepath = r"C:\Users\quinn\Desktop\CryptoProj\fingerprint_test.csv"
target_minutiae_count = 200

process_images_from_folder(folder_path, csv_filepath, target_minutiae_count)
#For single image "verification phase" set 'enrollment=False' to show enhanced image and minutiae results


The following code blocks contain features implemented in the above methods

In [3]:
"""
img = cv2.imread(r"C:\Users\quinn\Downloads\CrossMatch_Sample_DB\012_4_2.tif", 0)
out = fingerprint_enhancer.enhance_Fingerprint(img)
cv2.imshow('enhanced_image', out);
cv2.waitKey(0)
# You have to exit the pop up box to get the code block to finish

FeaturesTerminations, FeaturesBifurcations = fingerprint_feature_extractor.extract_minutiae_features(
            out, spuriousMinutiaeThresh=10, invertImage=False, showResult=True, saveResult=True)
# Exit the pop up

x=0
y=0
for i in FeaturesTerminations:
    i.Orientation = i.Orientation + [0,0]
    print(i.Orientation)
    x+=1
for i in FeaturesBifurcations:
    #i.Orientation = i.Orientation + [0,0]
    print(i.Orientation)
    y+=1
print("Terminations Length = ", x)
print("Bifurcations Length = ", y)
"""

-1